## DL Efron Benchmark

In [1]:
from xgbsurv.datasets import (load_metabric, load_flchain, load_rgbsg, load_support, load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs
from xgbsurv.models.utils import sort_X_y
import os
import numpy as np
import pandas as pd
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.decomposition import PCA
# import models
from xgbsurv.models.breslow_final import breslow_likelihood,get_cumulative_hazard_function_breslow
from xgbsurv.models.efron_final import efron_likelihood
from xgbsurv.models.cind_final import cind_loss
from xgbsurv.models.deephit_pycox_final import deephit_loss1_pycox
from xgbsurv.models.eh_aft_final import aft_likelihood
from xgbsurv.models.eh_ah_final import ah_likelihood
from pycox.evaluation import EvalSurv
from sklearn.utils.fixes import loguniform
from xgbsurv.models.utils import sort_X_y_pandas, transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
import os
import sys
current_path = os.getcwd() 
one_level_up = os.path.abspath(os.path.join(current_path,  ".."))
two_levels_up = os.path.abspath(os.path.join(current_path,  "..",".."))
sys.path.append(one_level_up+'/dl_pipeline')
from dl_pipeline import train_dl_complete

## Set Parameters

In [2]:
# set parameters
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10
early_stopping_rounds=10
base_score = 0.0
validation_size = 0.2
method = '_dl_'
model = 'efron'

# set seed for scipy
np.random.seed(rand_state)


## Custom Splitting

In [3]:
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)


## Training

In [4]:
data_set_fns = [load_metabric,  load_flchain, load_rgbsg, load_support] 
metrics_list = []

for idx, dataset in enumerate(data_set_fns):
    model = 'efron'
    # get name of current dataset
    data = dataset(path=two_levels_up+"/xgbsurv/datasets/data/", as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter)
        print(metric)
        metrics_list.append(metric)



METABRIC_adapted.csv
0
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5538073885956907
Integrated Brier Score: 0.1799889701606382
{'model': 'efron', 'dataset': 'METABRIC', 'cindex_train': [0.5863088619992655], 'cindex_test': [0.5538073885956907], 'ibs_train': [0.1644676411676665], 'ibs_score_test': [0.1799889701606382]}
1
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.6201301893842553
Integrated Brier Score: 0.16825008707395558
{'model': 'efron', 'dataset': 'METABRIC', 'cindex_train': [0.6195859914464766], 'cindex_test': [0.6201301893842553], 'ibs_train': [0.16661531392518442], 'ibs_score_test': [0.16825008707395558]}
2
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5821223955752037
Integrated Brier Score: 0.

In [5]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_results.csv',index=False)

In [6]:
df

,model,dataset,cindex_train,cindex_test,ibs_train,ibs_score_test
0,efron,METABRIC,0.586309,0.553807,0.164468,0.179989
1,efron,METABRIC,0.619586,0.620130,0.166615,0.168250
2,efron,METABRIC,0.582175,0.582122,0.169726,0.183975
3,efron,METABRIC,0.638851,0.625095,0.166306,0.160093
4,efron,METABRIC,0.617974,0.628429,0.165872,0.172739
5,efron,FLCHAIN,0.796920,0.803546,0.098728,0.100353
6,efron,FLCHAIN,0.782893,0.785078,0.102415,0.100260
7,efron,FLCHAIN,0.796362,0.783138,0.100779,0.105511
8,efron,FLCHAIN,0.791812,0.768447,0.102880,0.104944
9,efron,FLCHAIN,0.792659,0.795620,0.102004,0.096452


## TCGA Train, Test, Evaluation

In [7]:
cancer_types = [
    'BLCA',
    'BRCA',
    'HNSC',
    'KIRC',
    'LGG',
    'LIHC',
    'LUAD',
    'LUSC',
    'OV',
    'STAD'
    ]

metrics_list = []

for i, cancer_type in enumerate(cancer_types):
    model = 'efron'
    # get name of current dataset
    data = load_tcga(path=two_levels_up+"/xgbsurv/datasets/data/",cancer_type=cancer_type, as_frame=True)
    filename = data.filename
    X  = data.data #.astype(np.float32)
    y = data.target #.values #.to_numpy()

    print(filename)

    X, y = sort_X_y_pandas(X, y)
    dataset_name = filename.split('_')[0]
    
    for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
        print(i)
        metric = train_dl_complete(dataset_name, X, y, i,  train_index, test_index, model, n_iter, tcga=True)
        print(metric)
        metrics_list.append(metric)

BLCA_adapted.csv
0
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.5281767955801105
Integrated Brier Score: 0.22839483116280657
{'model': 'efron', 'dataset': 'BLCA', 'cindex_train': [0.535676094251263], 'cindex_test': [0.5281767955801105], 'ibs_train': [0.21226935226688998], 'ibs_score_test': [0.22839483116280657]}
1
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.4860731840524304
Integrated Brier Score: 0.21797082847140606
{'model': 'efron', 'dataset': 'BLCA', 'cindex_train': [0.5315154887775922], 'cindex_test': [0.4860731840524304], 'ibs_train': [0.214236132707433], 'ibs_score_test': [0.21797082847140606]}
2
model: efron
Fitting 5 folds for each of 10 candidates, totalling 50 fits
split type <class 'pandas.core.series.Series'>
Concordance Index 0.516726403823178
Integrated Brier Score: 0.2240663652494

In [8]:
df0 = pd.DataFrame(metrics_list)
df = df0.applymap(lambda x: x[0] if isinstance(x, list) else x)
df.to_csv(current_path+'/metrics/'+model+method+'_tcga_results.csv',index=False)